In [3]:
!mkdir weights
!wget https://github.com/nttcslab/m2d/releases/download/v0.1.0/msm_mae_vit_base-80x608p16x16-220924-mr75.zip \
  -O weights/msm_mae_vit_base-80x608p16x16-220924-mr75.zip

mkdir: cannot create directory ‘weights’: File exists


In [7]:
%cd weights
!unzip msm_mae_vit_base-80x608p16x16-220924-mr75.zip

[Errno 2] No such file or directory: 'weights'
/home/alc/TIL-2023/SpeakerID/m2d/weights
Archive:  msm_mae_vit_base-80x608p16x16-220924-mr75.zip
   creating: msm_mae_vit_base-80x608p16x16-220924-mr75/
  inflating: msm_mae_vit_base-80x608p16x16-220924-mr75/checkpoint-300.pth  


In [2]:
eval('evar.ar_m2d.AR_M2D')

NameError: name 'evar' is not defined

In [2]:
%cd ~/TIL-2023/SpeakerID/m2d
import sys
sys.path.append("./evar")

import torch
from pathlib import Path
from evar.ar_m2d import AR_M2D
from finetune import TaskNetwork
from lineareval import make_cfg
from evar.common import kwarg_cfg

device = torch.device("cuda")

cfg, n_folds, activation, balanced = make_cfg(
  config_file="evar/config/m2d.yaml",
  task="til",
  options="",
)
cfg.weight_file = "/home/alc/TIL-2023/SpeakerID/m2d/weights/msm_mae_vit_base-80x608p16x16-220924-mr75/checkpoint-300.pth"
cfg.unit_sec = 15.0
# change n_class according to no. of unique speakers
cfg.runtime_cfg = kwarg_cfg(n_class=2, hidden=())

run_path = Path("/home/alc/TIL-2023/SpeakerID/m2d/evar/logs/til_ar_m2d.AR_M2D_64f5263c")
state_dict = torch.load(run_path / "weights_ep0it0-0.66667_loss0.3365.pth")
norm_stats = torch.load(run_path / "norm_stats.pt")

backbone = AR_M2D(cfg, inference_mode=True, norm_stats=norm_stats).to(device)
model = TaskNetwork(cfg, ar=backbone).to(device)
model = torch.nn.DataParallel(model).to(device)

model.load_state_dict(state_dict)
model.eval()

/home/alc/TIL-2023/SpeakerID/m2d


/home/alc/venvs/til2023/lib/python3.10/site-packages/nnAudio/Spectrogram.py:4: Warning: importing Spectrogram subpackage will be deprecated soon. You should import the feature extractor from the feature subpackage. See actual documentation.
  warnings.warn(
/home/alc/venvs/til2023/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
ERROR:root:Install toprchopenl3.
>>> pip install torchopenl3
ERROR:root:Make your copy of VGGish under external folder. Check Preparing-models.md for the details.
ERROR:root:Install tensorflow and tensorflow_hub.
>>> pip install tensorflow tensorflow_hub
ERROR:root:Make your copy of COALA under external folder. Check Preparing-models.md for the details.



+task_metadata=evar/metadata/til.csv,+task_data=work/16k/til,+unit_samples=240000
MSM-MAE **FORCED DEC DEPTH AS 4 (Your decoder_depth=4 is ignored)**
MaskedAutoencoderViT(in_chans=1, patch size=(16, 16), grid_size=[ 5 38],
  embed_dim=768, depth=12, num_heads=12, decoder_embed_dim=384,
  decoder_depth=4, decoder_num_heads=6, mlp_ratio=4,
  norm_pix_loss=False)
 using 206 parameters, while dropped 0 from 206 parameters in /home/alc/TIL-2023/SpeakerID/m2d/weights/msm_mae_vit_base-80x608p16x16-220924-mr75/checkpoint-300.pth
3840 [] 2
Backbone representation:
Total number of parameters: 92,967,040 (trainable 92,747,008)
Trainable parameters: ['runtime.backbone.cls_token', 'runtime.backbone.mask_token', 'runtime.backbone.patch_embed.proj.weight', 'runtime.backbone.patch_embed.proj.bias', 'runtime.backbone.blocks.0.norm1.weight', 'runtime.backbone.blocks.0.norm1.bias', 'runtime.backbone.blocks.0.attn.qkv.weight', 'runtime.backbone.blocks.0.attn.qkv.bias', 'runtime.backbone.blocks.0.attn.pro

DataParallel(
  (module): TaskNetwork(
    (ar): AR_M2D(
      (runtime): RuntimeM2D(
        (backbone): MaskedAutoencoderViT(
          (patch_embed): PatchEmbed(
            (proj): Conv2d(1, 768, kernel_size=(16, 16), stride=(16, 16))
            (norm): Identity()
          )
          (blocks): ModuleList(
            (0-11): 12 x Block(
              (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
              (attn): Attention(
                (qkv): Linear(in_features=768, out_features=2304, bias=True)
                (attn_drop): Dropout(p=0.0, inplace=False)
                (proj): Linear(in_features=768, out_features=768, bias=True)
                (proj_drop): Dropout(p=0.0, inplace=False)
              )
              (ls1): Identity()
              (drop_path1): Identity()
              (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
              (mlp): Mlp(
                (fc1): Linear(in_features=768, out_features=3072, bias=True)
     

In [14]:
import torchaudio
import pickle
import torch

audio = torchaudio.load("/home/alc/TIL-2023/SpeakerID/m2d/evar/work/16k/til/valid-2.wav")[0]
classes_pickle_path = "/home/alc/TIL-2023/SpeakerID/m2d/evar/logs/til_ar_m2d.AR_M2D_64f5263c/classes.pkl"

with open(classes_pickle_path, "rb") as classes_pickle_file:
  classes = pickle.load(classes_pickle_file)

classes = list(classes)

model_output = model(audio)
pred_idx = torch.argmax(model_output[0])
pred_class_name = classes[pred_idx]

pred_class_name

'1'